# Control point: Modeling and validation
#### Máster en Análisis de Datos, Ciberseguridad y Computación en la Nube
#### Aprendizaje Automático - Punto de Control 2 (11/11/2019)

#### Complete name: 

## Introduction

This control point has two parts, one corresponding to a classification problem and the other one to a regression problem. You receive both parts at the same time. They do not necessarily take the same time for solving.

The main objective in each part of the control point is to solve a complete problem, including a soft preprocessing (necessary functions are provided below), model selection and model validation. 

You must take into account that the datasets are not toy datasets, and their sizes could be relevant.

Below you have auxiliary functions for preprocessing the data. They offer margin for slight adaptations, which could provide an extra pump to the scores you will get.

If you try anytime several options, it is important to show the results of those discarded trials, because what is not visible cannot be evaluated.

The deliverable of this control point is this Jupyter Notebook containing the code, plus some short answers in markdown cells if required. Comments inside the code are meant for helping understanding the code, not fot providing answers to the questions. Use markdown cells for that purpose.

NOTE: Keep in mind that some functions accept both Pandas dataframes and Numpy arrays, but some others only one of them. Nevertheless, we should know how to pass form one to the other and vice versa.

NOTE: Keep in mind that some functions will take some time to run. You can continue working on other cells during the run to avoid wasting time waiting.

REMINDER: Do not use python packages that we have not seen in the lecture.

REMINDER: The test must be written in ENGLISH. I will not consider any text or comment inside code if it is not written in English.

#### Auxiliar functions

The function **feature_selection_model** fits a SelectPercentile model on input (x) and target (y) using the selected score (default=chi2) and percentile (default=5).

The options for scores are: mutual_info_classif, mutual_info_regression, f_classif, f_regression and chi2.

In [112]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import mutual_info_classif, f_classif, chi2
from sklearn.feature_selection import mutual_info_regression, f_regression

def feature_selection_model(x, y, score=chi2, percentile=5):    
    fsm = SelectPercentile(score, percentile=percentile).fit(x, y)
    return fsm

The function **imbalance_correction_model_fitting** provides a way for applying undersampling, oversampling or mixed (oversampling + undersampling) strategies on some data, where we denote inputs as *x* and target as *y*, as well as a *model* that will be fit right after the correction on the balanced data. The parameter *strategy* (accepting values 'over', 'under' and 'mixed') determines the chosen strategy.

The options for undersampling are: 'EditedNearestNeighbours' and 'TomekLinks'. BorderlineSMOTE (version 1) is the only accepted overfitting technique, and will be used despite of the parameter you use in *over*.

The output is the model fitted for the provided data.

These days, the imbalanced-learn website was unavailable. In case you need to check the documentation, it is still available in the project github page (https://github.com/scikit-learn-contrib/imbalanced-learn/tree/master/doc).

In [2]:
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.pipeline import make_pipeline

def imbalance_correction_model_fitting(x, y, under, strategy, model, over='BorderlineSMOTE'): 
    over = BorderlineSMOTE(kind='borderline-1', random_state=seed)
    if under is 'TomekLinks':
        under = TomekLinks(random_state=seed)
    else:
        under = EditedNearestNeighbours(random_state=seed)
    if strategy is 'over':
        scheme = make_pipeline(over, model)
    elif strategy is 'under':
        scheme = make_pipeline(under, model)
    else:
        scheme = make_pipeline(over, under, model)
    
    fitted_model = scheme.fit(x, y)
    return fitted_model

## Exercises

### Part 1: Classification

The dataset 'sekholopane.csv' has 100 variables and almost 2000 samples. We will assume that it has neither outliers nor missing values. You have to perform the following tasks:
* (0) [0.25 points] Fix a seed for all random_state parameter. Use the sum of the digits of your DNI. Use this seed in all functions that have random_state parameter. 
<br><br>
* (1) [0.25 points] Split the data into train and test sets, keeping two thirds of the data for training. Make sure you keep the ratio of all the classes in both parts.
<br><br>
* (2) [1.5 points] Tune one of the SVC algorithms. Optimize the two more relevant parameters of the algorithm you have chosen, by means of 10-fold cross validation. Make sure that you check at least 17 different parameters combinations.
<br><br>
* (3) [1 points] Tune a neural network, optimizing the layers topology with at least 5 different topologies. Keep the rest of parameters with their default values, except random_state (where you must use your seed) and max_iter (use 1000).
<br><br>
* (4) [1 points] Tune a random forest algorithm, optimizing n_estimators and max_depth with at least 4 different values for both parameters. Comparing (4) with (2) and (3), which is the overall winning model? (Note: by winning model we mean the best paradigm + parameters. One example could be random forest with n_estimators=3 and max_depth=700).
<br><br>
* (5) [0.5 points] Would you say that the data is imbalanced? Looking at the available options for feature selection and for imbalance correction, how many combined (feature selection + imbalance correction) preprocessing schemes can you define?
<br><br>
* (6) [1 points] For the overall winning model, check whether performing any of the possible preprocessing schemes performs even better. 
<br><br>
* (7) [0.5 points] For the overall best model you have found in (6), obtain the classification report, the area under the ROC curve, and the confusion matrix.

### Solution

### [0.25 points] Fix a seed for all random_state parameter. Use the sum of the digits of your DNI. Use this seed in all functions that have random_state parameter.

In [3]:
# 72841579
seed = 43

### [0.25 points] Split the data into train and test sets, keeping two thirds of the data for training. Make sure you keep the ratio of all the classes in both parts. 

In [4]:
import pandas as pd

df = pd.read_csv('sekholopane.csv')

df.head()

,v1,v2,v3,v4,v5,v6,v7,v8,v9,v10,...,v92,v93,v94,v95,v96,v97,v98,v99,v100,class
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,-1.0
1,0.00,0.16,0.12,0.74,0.45,0.07,0.26,0.59,0.35,0.27,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,1.0
2,0.00,0.42,0.49,0.56,0.17,0.04,0.39,0.47,0.28,0.32,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,-1.0
3,0.04,0.77,1.00,0.08,0.12,0.10,0.51,0.50,0.34,0.21,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,-1.0
4,0.01,0.55,0.02,0.95,0.09,0.05,0.38,0.38,0.23,0.36,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,-1.0


In [5]:
from sklearn.model_selection import train_test_split

X = df.iloc[:,:-1]
y = df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed, stratify=y)

### [1.5 points] Tune one of the SVC algorithms. Optimize the two more relevant parameters of the algorithm you have chosen, by means of 10-fold cross validation. Make sure that you check at least 17 different parameters combinations. 

In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

tuned_parameters = {
        "random_state": [seed],
        "kernel": ["poly", "rbf", "sigmoid"],
        "gamma": ['auto','scale', 0.01, 0.1, 1, 10, 100]
}

clf = GridSearchCV(SVC(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)

print("best params: " + str(clf.best_params_))

y_pred = clf.predict(X_test)

score = roc_auc_score(y_test, y_pred)

print("roc_auc_score: " + str(score))

best params: {'gamma': 0.1, 'kernel': 'poly', 'random_state': 43}
roc_auc_score: 0.623431855500821


### [1 points] Tune a neural network, optimizing the layers topology with at least 5 different topologies. Keep the rest of parameters with their default values, except random_state (where you must use your seed) and max_iter (use 1000). 

In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

tuned_parameters = {
        "hidden_layer_sizes": [20,35,50,75,100,],        
        "random_state": [seed],
}

clf = GridSearchCV(MLPClassifier(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)
print("best params: " + str(clf.best_params_))


y_pred = clf.predict(X_test)
score = roc_auc_score(y_test, y_pred)
print("roc_auc_score: " + str(score))

best params: {'hidden_layer_sizes': 35, 'random_state': 43}
roc_auc_score: 0.7201477832512315


### [1 points] Tune a random forest algorithm, optimizing n_estimators and max_depth with at least 4 different values for both parameters. Comparing (4) with (2) and (3), which is the overall winning model? (Note: by winning model we mean the best paradigm + parameters. One example could be random forest with n_estimators=3 and max_depth=700). 

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

tuned_parameters = {
    "n_estimators": [10, 50, 100, 150],
    "max_depth": [1, 10, 100, 1000],
    "random_state": [seed],    
}

clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5)
clf.fit(X_train, y_train)
print("best params: " + str(clf.best_params_))


y_pred = clf.predict(X_test)
score = roc_auc_score(y_test, y_pred)
print("roc_auc_score: " + str(score))

best params: {'max_depth': 100, 'n_estimators': 100, 'random_state': 43}
roc_auc_score: 0.6450738916256158


We get the best score (0.72) with Neural Network. With the following params:

MLPClassifier(hidden_layer_sizes=35, random_state=43)

### [0.5 points] Would you say that the data is imbalanced? Looking at the available options for feature selection and for imbalance correction, how many combined (feature selection + imbalance correction) preprocessing schemes can you define? 

In [9]:
y.value_counts()

-1.0    1844
 1.0     150
Name: class, dtype: int64

Yes, data is inbalanced!

In [10]:
df.isnull().values.any()

False

We dont have any missing values.

###### FEATURE SELECTION (SELECT PERCENTILE)
para ir avanzando mas rapido, he decidido dejar sin hacer esta parte.


### [1 points] For the overall winning model, check whether performing any of the possible preprocessing schemes performs even better.

In [22]:
from sklearn.neural_network import MLPClassifier

def score_with_best_model(X_train, X_test, y_train, y_test):
    model = MLPClassifier(hidden_layer_sizes=35, random_state=seed)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(str(roc_auc_score(y_test, y_pred)))    
    

We will try NearMiss undersampling technique to try to balance the data

We will try to select most relevant features using all the dataset.

In [27]:
# we maintain 20% of features
X_new = pd.DataFrame(feature_selection_model(X, y, score=mutual_info_classif, percentile=20).transform(X))
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_new, y, test_size=0.33, random_state=seed, stratify=y)
X_new.shape

(1994, 75)

In [28]:
score_with_best_model(X_train1, X_test1, y_train1, y_test1)

0.6817898193760262


After trying with different percentiles we dont get better score so we are going to continue with all features.

### [0.5 points] For the overall best model you have found in (6), obtain the classification report, the area under the ROC curve, and the confusion matrix.

In [30]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

model = MLPClassifier(hidden_layer_sizes=35, random_state=seed)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [31]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        -1.0       0.96      0.98      0.97       609
         1.0       0.66      0.46      0.54        50

    accuracy                           0.94       659
   macro avg       0.81      0.72      0.75       659
weighted avg       0.93      0.94      0.94       659



In [34]:
print('roc_auc_score: '  + str(roc_auc_score(y_test, y_pred)))   

roc_auc_score: 0.7201477832512315


In [36]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred))

[[597  12]
 [ 27  23]]


### Part 2: Regression

The dataset 'superconducting.csv' has 81 variables and more than 21K samples. It is a real-world dataset about predicting the critical temperature of semiconducting materials.

* (1) [0.25 points] Split the data into train, validation and test sets, keeping 20% for both validation and test.
<br><br>
* (2) [1 points] Using the validation data and the mean squared error score in principal components regression (PCR), find the adequate number, $a$, of principal components, up to a maximum of 40. Focusing now on the cummulative variance, is the cummulative variance you are not capturing (because of not selecting the last $81 - a$ PCs) higher than 1 per 1000?
<br><br>
* (3) [1 points] Now we will check the performance using feature selection, instead of the feature extraction by PCA done in (2). Try all suitable scoring options of the feature_selection_model function, fixing percentile=5. Use multiple linear regression as predictive model. Which compression has been higher?
<br><br>
* (4) [1 points] Repeat (2) and (3) using ridge regression instead of multiple linear regression. Set the best of all models seen so far, as your winning model.
<br><br>
* (5) [0.75 points] Check the performance of the winning model on the test set. Compare it with the performance on the original dataset (without preprocessing) using the same predictor as in the winning model (multiple linear regression or ridge regression). Was our preprocessing beneficial?

### Solution

In [91]:
import pandas as pd

seed = 43

df = pd.read_csv('superconducting.csv')

df.head()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0
1,5,92.729214,58.518416,73.132787,36.396602,1.449309,1.057755,122.90607,36.161939,47.094633,...,2.257143,1.888175,2.210679,1.557113,1.047221,2,1.128571,0.632456,0.468606,26.0
2,4,88.944468,57.885242,66.361592,36.122509,1.181795,0.975980,122.90607,35.741099,51.968828,...,2.271429,2.213364,2.232679,1.368922,1.029175,1,1.114286,0.433013,0.444697,19.0
3,4,88.944468,57.873967,66.361592,36.119560,1.181795,1.022291,122.90607,33.768010,51.968828,...,2.264286,2.213364,2.226222,1.368922,1.048834,1,1.100000,0.433013,0.440952,22.0
4,4,88.944468,57.840143,66.361592,36.110716,1.181795,1.129224,122.90607,27.848743,51.968828,...,2.242857,2.213364,2.206963,1.368922,1.096052,1,1.057143,0.433013,0.428809,23.0


### [0.25 points] Split the data into train, validation and test sets, keeping 20% for both validation and test. 

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

### [1 points] Using the validation data and the mean squared error score in principal components regression (PCR), find the adequate number, 𝑎, of principal components, up to a maximum of 40. Focusing now on the cummulative variance, is the cummulative variance you are not capturing (because of not selecting the last 81−𝑎 PCs) higher than 1 per 1000? 

In [100]:
import pandas as pd
from sklearn.decomposition import PCA

# Calculates dataframe PCA
def get_pca(X, pca): 
    pca = PCA(n_components=pca)
    pca.fit(X)
    X_reduced = pca.transform(X)
    # print("There have been selected " + str(X_reduced.shape[1]) + " principal components.")    
    columns = []
    for n in range(X_reduced.shape[1]):
        columns.append("PCA" + str(n))    
    df = pd.DataFrame(X_reduced, columns=columns)
    return df  

X_train_pca = get_pca(X_train, 0.9)

X_train_pca.head()

,PCA0,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,...,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20
0,-0.312360,-1.170030,0.208600,0.153494,0.362203,-0.162382,0.261143,-0.206815,0.361993,0.052164,...,0.159684,-0.313973,-0.063068,-0.184779,-0.114812,0.035941,-0.060049,0.126594,0.059511,0.028170
1,0.855798,0.353396,-0.316044,1.671412,-0.025749,0.793829,0.712299,0.586164,-0.530098,0.576396,...,-0.003891,-0.348147,0.041137,0.306338,0.205001,0.239133,0.164228,0.188800,0.102803,0.184826
2,-0.448908,0.339928,-0.855757,1.965854,-0.291555,0.462020,0.813965,0.330929,-0.495265,0.319464,...,-0.003597,-0.021071,-0.290377,0.259515,0.396133,-0.100374,0.057554,0.099562,0.098758,-0.200112
3,-0.247560,-0.356068,0.464626,0.712587,0.072139,-0.259296,-0.068752,-0.282230,0.199696,-0.020204,...,0.008447,-0.205174,0.255761,0.046238,-0.078202,-0.096612,-0.066516,-0.162972,-0.057491,0.135162
4,-0.537044,-0.329914,-0.586943,0.251302,-0.326870,-0.432467,-0.005168,-0.639395,-0.122173,-0.202462,...,0.003045,0.278517,-0.193945,0.220143,0.306393,0.058638,0.184288,-0.029122,0.126803,-0.016718


With 0.90, we get 21 principal components

In [101]:
components = 21

In [102]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression().fit(X_train_pca, y_train)
y_pred = model.predict(get_pca(X_val, components))
error = mean_squared_error(y_pred, y_val)

print("mean_squared_error (PCA): " + str(error))

mean_squared_error (PCA): 0.23120946360009703


### [1 points] Now we will check the performance using feature selection, instead of the feature extraction by PCA done in (2). Try all suitable scoring options of the feature_selection_model function, fixing percentile=5. Use multiple linear regression as predictive model. Which compression has been higher? 

In [103]:
percentile = 20

##### chi2

In [106]:
model = feature_selection_model(X_train, y_train, percentile=percentile)
X_train_selec = model.transform(X_train)
X_val_selec = model.transform(X_val)
print(X_train_selec.shape)
print(X_val_selec.shape)

(1276, 20)
(319, 20)


We reduced to 15 features 

In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression().fit(X_train_selec, y_train)
y_pred = model.predict(X_val_selec)
error = mean_squared_error(y_pred, y_val)
print("mean_squared_error (chi2): " + str(error))

mean_squared_error (chi2): 0.16510738819304246


Knowing that the best score (error) we can get is 0, we get better score selection %20 of features with chi2 than  using 21 PCAs.

### [1 points] Repeat (2) and (3) using ridge regression instead of multiple linear regression. Set the best of all models seen so far, as your winning model. 

In [109]:
from sklearn.linear_model import Ridge

pca_model = Ridge(alpha=1.0).fit(X_train_pca, y_train)
y_pred = model.predict(get_pca(X_val, components)) # tiene la misma cantidad de features que en X_train_pca
                                                    # no se porque me da error de dimensiones
error = mean_squared_error(y_pred, y_val)

print("mean_squared_error(PCA+Ridge): " + str(error))

ValueError: shapes (319,21) and (20,) not aligned: 21 (dim 1) != 20 (dim 0)

In [110]:
from sklearn.linear_model import Ridge

feature_selection_model = Ridge(alpha=1.0).fit(X_train_selec, y_train)
y_pred = model.predict(X_val_selec)
error = mean_squared_error(y_pred, y_val)
print("mean_squared_error (chi2+Ridge): " + str(error))

mean_squared_error (chi2+Ridge): 0.16510738819304246


### [0.75 points] Check the performance of the winning model on the test set. Compare it with the performance on the original dataset (without preprocessing) using the same predictor as in the winning model (multiple linear regression or ridge regression). Was our preprocessing beneficial?

Como me ha dado error al usar PCA + Ridge, voy a asumir para el mejor son los datos seleccionados con Chi2, usandolo en este ejercicio.

In [117]:
import numpy as np

X_train_val = X_train.copy()
X_train_val = X_train_val.append(X_val)

y_train_val = y_train.copy()
y_train_val = y_train_val.append(y_val)

model = Ridge(alpha=1.0).fit(X_train_val, y_train_val)

y_pred = model.predict(X_test)
error = mean_squared_error(y_pred, y_test)
print("mean_squared_error (original data): " + str(error))

mean_squared_error (original data): 0.1911301686258577
